<a href="https://colab.research.google.com/github/realyou0312/KPS_keyword_project/blob/main/1112%EC%B9%B4%EC%9A%B4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install urllib3==1.24.1

In [ ]:
import requests
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
def web_request(method_name, url, is_urlencoded=True, timeout_seconds=3):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper() # 메소드이름을 대문자로 바꾼다 
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')
        
    if method_name == 'GET': # GET방식인 경우
        response = requests.get(url=url, timeout=timeout_seconds)
    elif method_name == 'POST': # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, \
                                     timeout=timeout_seconds, headers=header)
        else:
            response = requests.post(url=url, \
                                     timeout=timeout_seconds, headers=header)
    return response
    
def web_request_retry(num_retry=3, sleep_seconds=1, **kwargs):
    """timeout발생 시 sleep_seconds쉬고 num_retyrp번 재시도 한다"""
    for n in range(num_retry):
        try:
            return web_request(**kwargs)
        except:
          time.sleep(sleep_seconds)
          continue
        # except requests.exceptions.Timeout:
        #     print(str(n+1) + ' Timeout')
        #     time.sleep(sleep_seconds)
        #     continue
        # except requests.exceptions.ConnectionError:
        #     time.sleep(sleep_seconds)
        #     print("Connection Error!!!!!!!!")
        #     continue
    return None

def checkCodeName(code, code_name):
  url = f'https://api.deepsearch.com/v1/compute?input=FindEntityByName("Financial","{code_name}")'
  res = web_request_retry(method_name='POST', url=url, is_urlencoded=True, num_retry=5 )
  res_data = res.json()

  try:
    if code in res_data['data']['pods'][1]['content']['data']['symbol']:
      return True
    else:
      return False
  except:
    return False


In [ ]:
header = {'Authorization':'Basic {ZTY1MTAzOTc4MWZjNDg2OTgyNDQ2NDcyMGEzMjQ3M2M6MWNjYjUxYjdiZmIwNTdhMDJjYTllMGIyZmVjZmRlNmFiZjM0YjVkZDlhZTk2YjQ3Y2QwYzIwZTRkMWEyODRiZA==}'}

In [ ]:
path = '/content/drive/MyDrive/KAKAOPAYSEC/KR_stocks4.csv'

In [ ]:
stock = pd.read_csv(path, encoding='utf-8')
data_list = stock.values.tolist()

In [ ]:
stock

,종목코드,종목이름,거래량
0,005930,삼성전자,2.580000e+14
1,068270,셀트리온,1.140000e+14
2,035720,카카오,1.120000e+14
3,019170,신풍제약,1.060000e+14
4,011200,HMM,1.000000e+14
...,...,...,...
2521,245450,씨앤에스링크,2.067701e+08
2522,217320,썬테크,1.249014e+08
2523,179720,렌딩머신,1.023674e+08
2524,225850,미애부,8.222744e+07


In [ ]:
data_list

[['005930', '삼성전자', 258000000000000.0],
 ['068270', '셀트리온', 114000000000000.0],
 ['035720', '카카오', 112000000000000.0],
 ['019170', '신풍제약', 106000000000000.0],
 ['011200', 'HMM', 100000000000000.0],
 ['005380', '현대차', 96000000000000.0],
 ['091990', '셀트리온헬스케어', 93900000000000.0],
 ['000660', 'SK하이닉스', 86900000000000.0],
 ['066570', 'LG전자', 84800000000000.0],
 ['000270', '기아', 82700000000000.0],
 ['302440', 'SK바이오사이언스', 73500000000000.0],
 ['096770', 'SK이노베이션', 73000000000000.0],
 ['323990', '박셀바이오', 66800000000000.0],
 ['035420', 'NAVER', 62100000000000.0],
 ['096530', '씨젠', 61300000000000.0],
 ['051910', 'LG화학', 61200000000000.0],
 ['011000', '진원생명과학', 59100000000000.0],
 ['048410', '현대바이오', 55000000000000.0],
 ['034020', '두산중공업', 54900000000000.0],
 ['068760', '셀트리온제약', 53500000000000.0],
 ['006400', '삼성SDI', 53300000000000.0],
 ['009830', '한화솔루션', 45500000000000.0],
 ['041190', '우리기술투자', 45300000000000.0],
 ['323410', '카카오뱅크', 45300000000000.0],
 ['293490', '카카오게임즈', 45100000000000.0]

In [ ]:
# 총 키워드 뽑기
result_list = []

for data in data_list:
  code = "KRX:" + data[0].zfill(6)
  name = data[1]
  if name == '와이솔':
    try:
      # 총 키워드 수 뽑기
        url = f'https://api.deepsearch.com/v1/compute?input=DocumentAggregation("news","economy","{code}","named_entities.entities.brand.name:10000",date_from=2010-01-01)'
        res_url = web_request_retry(method_name='POST', url=url, is_urlencoded=True, num_retry=5)
        res_data = res_url.json()

        result = res_data['data']['pods'][1]['content']['data']['key']
        count = res_data['data']['pods'][1]['content']['data']['count']

        df = pd.DataFrame(data=[result, count], index=['keyword', 'count'])
        df = df.transpose()
        df['count'].astype(int)

        # df = df[df['count'] > 2]

        # df.to_csv(f'/content/drive/MyDrive/KAKAOPAYSEC/1112카운트/{name}.csv', index=False, encoding='utf-8-sig')


    except:
      print("total_keyword error!!!!!!!!!!!       " + name)
      result = []

for i in result:
  print(i)

갤럭시S3
갤노트7
갤럭시S6
갤럭시S8
갤럭시
갤럭시노트
아이폰
갤럭시S
갤럭시 S6
갤럭시 S7
갤럭시S4
갤3
갤S8
갤럭시 노트
갤럭시S10
갤럭시S20
갤럭시S7
갤럭시S9
갤럭시노트7
아이폰5
갤S3
갤S5
갤S7
갤노7
갤노트10
갤노트8
갤노트9
갤럭시 S10
갤럭시 S20
갤럭시 S8
갤럭시 노트4
갤럭시 쇼크
갤럭시3S
갤럭시8
갤럭시S2
갤럭시S6향
갤럭시S7향
갤럭시Z
갤럭시노트10
갤럭시노트5
갤럭시노트8
갤럭시노트株
갤럭시폴드2
기가헤르츠
노트7
비에이치
아이폰11
아이폰X
와이솔
와이지-원
파트론


In [ ]:
url = f'https://api.deepsearch.com/v1/compute?input=DocumentAggregation("news","economy","KRX:005930","named_entities.entities.brand.name:100000",date_from=2010-01-01)'
res_url = web_request_retry(method_name='POST', url=url, is_urlencoded=True, num_retry=5)
res_data = res_url.json()

result = res_data['data']['pods'][1]['content']['data']['key']

In [ ]:
count = res_data['data']['pods'][1]['content']['data']['count']

In [ ]:
df = pd.DataFrame(data=[result, count], index=['keyword', 'count'])
df = df.transpose()
df['count'].astype(int)

0       7531
1       2560
2       2122
3       1947
4       1449
        ... 
2426       1
2427       1
2428       1
2429       1
2430       1
Name: count, Length: 2431, dtype: int64

In [ ]:
df = df[df['count'] > 2]
